In [2]:
print("""Question 1. Formulate this problem as an LP in Pulp. Name your decision variables and constraints clearly.
Output the model formulation (objective function and the constraints), Solve the model, 
then output the optimal solution, the basic variables and the total cost.""")
from pulp import *
import pandas

# row numbers are plants column numbers are distributors
c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
# row numbers are plants, column numbers are retailers
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
# row numbers are distributors, column numbers are retailers
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 11, 11]]
plant_capacities = [137, 266, 104, 364]
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)
# Plant, distributor and retailer naming start with 0. This is consistent with Python indices.
# You can use the makeDict() function of Pulp to prepare data for the LP Models
problem = LpProblem("Optimal_Logistics", LpMinimize) # beacuse we want to minimize the transportation and transshipment costs 
# decision_variables
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0) # number of units to send from plants to distributors
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) # number of units to send from plants to retailers
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) # number of units to send from distributors to retailers

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}

# adding obj function to the problem
problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)
# adding constraints to the problem
# this is for not exceeding plant capacities
for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
# indicates total number of products sent from plants to a distributors must be equal to total number of products sent from distributors to retailers
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
# indicates total number of products sent from plants and from distributors to one retailer must satisfy this retailer's demand.
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE ORIGINAL PROBLEM MODELED AS AN LP:")
print("")
print(problem)
problem.solve()
# printing the status of the problem: optimal, infeasible etc.
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
# Remember: if slack = 0, then constraint is binding and changing binding constraint, changes the solution.
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)

Question 1. Formulate this problem as an LP in Pulp. Name your decision variables and constraints clearly.
Output the model formulation (objective function and the constraints), Solve the model, 
then output the optimal solution, the basic variables and the total cost.

THIS IS THE ORIGINAL PROBLEM MODELED AS AN LP:

Optimal_Logistics:
MINIMIZE
12*X_d_r_0_0 + 18*X_d_r_0_1 + 17*X_d_r_0_2 + 15*X_d_r_0_3 + 16*X_d_r_1_0 + 13*X_d_r_1_1 + 13*X_d_r_1_2 + 10*X_d_r_1_3 + 15*X_d_r_2_0 + 13*X_d_r_2_1 + 11*X_d_r_2_2 + 11*X_d_r_2_3 + 15*X_p_d_0_0 + 25*X_p_d_0_1 + 14*X_p_d_0_2 + 20*X_p_d_1_0 + 18*X_p_d_1_1 + 11*X_p_d_1_2 + 14*X_p_d_2_0 + 12*X_p_d_2_1 + 21*X_p_d_2_2 + 26*X_p_d_3_0 + 15*X_p_d_3_1 + 28*X_p_d_3_2 + 68*X_p_r_0_0 + 46*X_p_r_0_1 + 35*X_p_r_0_2 + 58*X_p_r_0_3 + 57*X_p_r_1_0 + 69*X_p_r_1_1 + 58*X_p_r_1_2 + 60*X_p_r_1_3 + 60*X_p_r_2_0 + 69*X_p_r_2_1 + 36*X_p_r_2_2 + 61*X_p_r_2_3 + 54*X_p_r_3_0 + 40*X_p_r_3_1 + 49*X_p_r_3_2 + 35*X_p_r_3_3 + 0
SUBJECT TO
_C1: 2 X_p_r_0_0 + 2 X_p_r_0_1 + 2 X_p_r

In [3]:
print(""" Question 2. Answer these questions according to the results of question 1 and seperately for each task. 
Firsly argue your expectations with reasoning using economic interpretation of the results without resolving the LP,
then reformulate a seperate model and solve it, inspect the total cost and confirm your expectations.
Do not forget to update the data for each task. Note that the naming of plants, distributors and retailers start with 0.

    Question 2.a. The capacity of plant 3 increases by one unit.
    What is the expected change in the total cost? Confirm your expectations with resolving the LP.""")

print(""" Answer 2.a.: Shadow price is if capacity increases by one unit how does the objective function value changes. 
Because Plant 3's capacity is in constraint 4 and shadow price is indicated as 0 in the terminal, this constraint is nonbinding, objective function and optimal value doesn't change.""")
c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 11, 11]]
plant_capacities = [137, 266, 104, 365] # 3rd index was 364, it is increased by 1
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)
problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0) 
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0)
c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}
retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}

# adding obj function to the problem
problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)
# adding constraints to the problem
# this is for not exceeding plant capacities
for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
# indicates total number of products sent from plants to a distributors must be equal to total number of products sent from distributors to retailers
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
# indicates total number of products sent from plants and from distributors to one retailer must satisfy this retailer's demand.
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE PROBLEM AFTER THE CHANGES IN QUESTION 2.a.:")
print("")
print(problem)
problem.solve()
# printing the status of the problem: optimal, infeasible etc.
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
# Remember: if slack = 0, then constraint is binding and changing binding constraint, changes the solution.
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)


 Question 2. Answer these questions according to the results of question 1 and seperately for each task. 
Firsly argue your expectations with reasoning using economic interpretation of the results without resolving the LP,
then reformulate a seperate model and solve it, inspect the total cost and confirm your expectations.
Do not forget to update the data for each task. Note that the naming of plants, distributors and retailers start with 0.

    Question 2.a. The capacity of plant 3 increases by one unit.
    What is the expected change in the total cost? Confirm your expectations with resolving the LP.
 Answer 2.a.: Shadow price is if capacity increases by one unit how does the objective function value changes. 
Because Plant 3's capacity is in constraint 4 and shadow price is indicated as 0 in the terminal, this constraint is nonbinding, objective function and optimal value doesn't change.

THIS IS THE PROBLEM AFTER THE CHANGES IN QUESTION 2.a.:

Optimal_Logistics:
MINIMIZE
12*X_d_r

In [4]:
print("""Question 2.b. The capacity of plant 0 increases by one unit.
    What is the expected change in the total cost? Confirm your expectations with resolving the LP.""")
print("""Answer 2.b.:
      the constraint about capacity of plant 0 is constraint 1. Its shadow price is 0 means that 
      doesn't effect the objective function and its optimal value.""")

c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 11, 11]]
plant_capacities = [138, 266, 104, 364] # 0th index was 137, it is increased by 1
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)
problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0) 
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0)
c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}
retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}

# adding obj function to the problem
problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)
# adding constraints to the problem
# this is for not exceeding plant capacities
for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
# indicates total number of products sent from plants to a distributors must be equal to total number of products sent from distributors to retailers
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
# indicates total number of products sent from plants and from distributors to one retailer must satisfy this retailer's demand.
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE PROBLEM AFTER THE CHANGES IN QUESTION 2.b.:")
print("")
print(problem)
problem.solve()
# printing the status of the problem: optimal, infeasible etc.
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
# Remember: if slack = 0, then constraint is binding and changing binding constraint, changes the solution.
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)


Question 2.b. The capacity of plant 0 increases by one unit.
    What is the expected change in the total cost? Confirm your expectations with resolving the LP.
Answer 2.b.:
      the constraint about capacity of plant 0 is constraint 1. Its shadow price is 0 means that 
      doesn't effect the objective function and its optimal value.

THIS IS THE PROBLEM AFTER THE CHANGES IN QUESTION 2.b.:

Optimal_Logistics:
MINIMIZE
12*X_d_r_0_0 + 18*X_d_r_0_1 + 17*X_d_r_0_2 + 15*X_d_r_0_3 + 16*X_d_r_1_0 + 13*X_d_r_1_1 + 13*X_d_r_1_2 + 10*X_d_r_1_3 + 15*X_d_r_2_0 + 13*X_d_r_2_1 + 11*X_d_r_2_2 + 11*X_d_r_2_3 + 15*X_p_d_0_0 + 25*X_p_d_0_1 + 14*X_p_d_0_2 + 20*X_p_d_1_0 + 18*X_p_d_1_1 + 11*X_p_d_1_2 + 14*X_p_d_2_0 + 12*X_p_d_2_1 + 21*X_p_d_2_2 + 26*X_p_d_3_0 + 15*X_p_d_3_1 + 28*X_p_d_3_2 + 68*X_p_r_0_0 + 46*X_p_r_0_1 + 35*X_p_r_0_2 + 58*X_p_r_0_3 + 57*X_p_r_1_0 + 69*X_p_r_1_1 + 58*X_p_r_1_2 + 60*X_p_r_1_3 + 60*X_p_r_2_0 + 69*X_p_r_2_1 + 36*X_p_r_2_2 + 61*X_p_r_2_3 + 54*X_p_r_3_0 + 40*X_p_r_3_1 + 49*X_

In [5]:
print("""   Question 2.c. The demand of retailer 0 increases by one unit.
    What is the expected change in the total cost? Confirm your expectations with resolving the LP.""")
print("""Answer 2.c.:
      In this case, I have to update the demand quantity of retailer 0. Because C8 is about retailer 0's demand, we have to look at its shadow price.
# Since its shadow price is 26 in the original question, our optimal objective value should increase by 26.""")

c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 11, 11]]
plant_capacities = [137, 266, 104, 364]
demand_quantities = [297, 211, 104, 117] # 0th index is increased by one
plants = range(4)
distributors = range(3)
retailers = range(4)

problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0)
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) 

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}


problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)

for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE LP AFTER THE CHANGES IN 2.c.:")
print("")
print(problem)
problem.solve()
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)

   Question 2.c. The demand of retailer 0 increases by one unit.
    What is the expected change in the total cost? Confirm your expectations with resolving the LP.
Answer 2.c.:
      In this case, I have to update the demand quantity of retailer 0. Because C8 is about retailer 0's demand, we have to look at its shadow price.
# Since its shadow price is 26 in the original question, our optimal objective value should increase by 26.

THIS IS THE LP AFTER THE CHANGES IN 2.c.:

Optimal_Logistics:
MINIMIZE
12*X_d_r_0_0 + 18*X_d_r_0_1 + 17*X_d_r_0_2 + 15*X_d_r_0_3 + 16*X_d_r_1_0 + 13*X_d_r_1_1 + 13*X_d_r_1_2 + 10*X_d_r_1_3 + 15*X_d_r_2_0 + 13*X_d_r_2_1 + 11*X_d_r_2_2 + 11*X_d_r_2_3 + 15*X_p_d_0_0 + 25*X_p_d_0_1 + 14*X_p_d_0_2 + 20*X_p_d_1_0 + 18*X_p_d_1_1 + 11*X_p_d_1_2 + 14*X_p_d_2_0 + 12*X_p_d_2_1 + 21*X_p_d_2_2 + 26*X_p_d_3_0 + 15*X_p_d_3_1 + 28*X_p_d_3_2 + 68*X_p_r_0_0 + 46*X_p_r_0_1 + 35*X_p_r_0_2 + 58*X_p_r_0_3 + 57*X_p_r_1_0 + 69*X_p_r_1_1 + 58*X_p_r_1_2 + 60*X_p_r_1_3 + 60*X_p_r_2_0

In [6]:
print("""Question 2.d. The cost of sending one unit from distributor 2 to retailer 2 decreases by one.
Do you expect a change in the basis? What is the expected change in the cost? Confirm your expectations with resolving the LP.""")
print("""Answer 2.d.: It will change our c_d_r_l matrix and our objective function.
Basis can change as the cost coefficient of the decision variable related to the changed cost has changed. 
Since the cost has decreased, we can anticipate a decrease in the total cost by an amount equal to the decrease in the cost coefficient 
multiplied by optimal value of the related decision variable X_d_r_2_2 which is 1*104.
Since X_d_r_2_2 is at the basis at optimal, it will keep staying at basis and the basis will remain the same.""")

c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 10, 11]] # its 2nd index's 2nd index is decreased by 1.
plant_capacities = [137, 266, 104, 364]
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)

problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0)
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) 

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}


problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)

for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE LP AFTER THE CHANGES IN 2.d.:")
print("")
print(problem)
problem.solve()
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)

Question 2.d. The cost of sending one unit from distributor 2 to retailer 2 decreases by one.
Do you expect a change in the basis? What is the expected change in the cost? Confirm your expectations with resolving the LP.
Answer 2.d.: It will change our c_d_r_l matrix and our objective function.
Basis can change as the cost coefficient of the decision variable related to the changed cost has changed. 
Since the cost has decreased, we can anticipate a decrease in the total cost by an amount equal to the decrease in the cost coefficient 
multiplied by optimal value of the related decision variable X_d_r_2_2 which is 1*104.
Since X_d_r_2_2 is at the basis at optimal, it will keep staying at basis and the basis will remain the same.

THIS IS THE LP AFTER THE CHANGES IN 2.d.:

Optimal_Logistics:
MINIMIZE
12*X_d_r_0_0 + 18*X_d_r_0_1 + 17*X_d_r_0_2 + 15*X_d_r_0_3 + 16*X_d_r_1_0 + 13*X_d_r_1_1 + 13*X_d_r_1_2 + 10*X_d_r_1_3 + 15*X_d_r_2_0 + 13*X_d_r_2_1 + 10*X_d_r_2_2 + 11*X_d_r_2_3 + 15*X_p_d_0

In [7]:
print("""    Question 2.e. The cost of sending one unit from distributor 2 to retailer 3 decreases by one.
    Do you expect a change in the basis? What is the expected change in the cost? Confirm your expectations with resolving the LP.""")
print("""Answer 2.e.: It will change our c_d_r_l matrix and our objective function X_d_r_2_3's reduced cost is 0 and it is not in the basis,
a reduced cost of 0 for a non-basic variable implies the variable could enter the basis.
If we look at change in cost, it will decrease by decrease amount multiplied by related decision variable's 
optimal value which is 1*117.""")

c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 11, 10]] # Its second index's 3rd index is decreased by one. 
plant_capacities = [137, 266, 104, 364]
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)

problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0)
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) 

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}


problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)

for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE LP AFTER THE CHANGES IN 2.e.:")
print("")
print(problem)
problem.solve()
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)

    Question 2.e. The cost of sending one unit from distributor 2 to retailer 3 decreases by one.
    Do you expect a change in the basis? What is the expected change in the cost? Confirm your expectations with resolving the LP.
Answer 2.e.: It will change our c_d_r_l matrix and our objective function X_d_r_2_3's reduced cost is 0 and it is not in the basis,
a reduced cost of 0 for a non-basic variable implies the variable could enter the basis.
If we look at change in cost, it will decrease by decrease amount multiplied by related decision variable's 
optimal value which is 1*117.

THIS IS THE LP AFTER THE CHANGES IN 2.e.:

Optimal_Logistics:
MINIMIZE
12*X_d_r_0_0 + 18*X_d_r_0_1 + 17*X_d_r_0_2 + 15*X_d_r_0_3 + 16*X_d_r_1_0 + 13*X_d_r_1_1 + 13*X_d_r_1_2 + 10*X_d_r_1_3 + 15*X_d_r_2_0 + 13*X_d_r_2_1 + 11*X_d_r_2_2 + 10*X_d_r_2_3 + 15*X_p_d_0_0 + 25*X_p_d_0_1 + 14*X_p_d_0_2 + 20*X_p_d_1_0 + 18*X_p_d_1_1 + 11*X_p_d_1_2 + 14*X_p_d_2_0 + 12*X_p_d_2_1 + 21*X_p_d_2_2 + 26*X_p_d_3_0 + 15*X_p_d_

In [8]:
print("""Question 2.f. What if the decrease in Question 2.e. was 2?
    Do you expect a change in the basis? What is the change in the cost? Confirm your expectations with resolving the LP.""")
print("""Answer 2.f.: if the decrease was 2, it will remain in the basis and basis won't change 
      but the optimal cost will decrease by 2*117 which is 254.""")
print("""    Question 2.e. The cost of sending one unit from distributor 2 to retailer 3 decreases by one.
    Do you expect a change in the basis? What is the expected change in the cost? Confirm your expectations with resolving the LP.""")
print("""Answer 2.e.: It will change our c_d_r_l matrix and our objective function X_d_r_2_3's reduced cost is 0 and it is not in the basis,
a reduced cost of 0 for a non-basic variable implies the variable could enter the basis.
If we look at change in cost, it will decrease by decrease amount multiplied by related decision variable's 
optimal value which is 1*117.""")

c_p_d_l = [
[15, 25, 14],
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15],
[16, 13, 13, 10],
[15, 13, 11, 9]] # Its second index's 3rd index is decreased by two. 
plant_capacities = [137, 266, 104, 364]
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)

problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0)
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) 

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}


problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)

for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE LP AFTER THE CHANGES IN 2.f.:")
print("")
print(problem)
problem.solve()
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)

Question 2.f. What if the decrease in Question 2.e. was 2?
    Do you expect a change in the basis? What is the change in the cost? Confirm your expectations with resolving the LP.
Answer 2.f.: if the decrease was 2, it will remain in the basis and basis won't change 
      but the optimal cost will decrease by 2*117 which is 254.
    Question 2.e. The cost of sending one unit from distributor 2 to retailer 3 decreases by one.
    Do you expect a change in the basis? What is the expected change in the cost? Confirm your expectations with resolving the LP.
Answer 2.e.: It will change our c_d_r_l matrix and our objective function X_d_r_2_3's reduced cost is 0 and it is not in the basis,
a reduced cost of 0 for a non-basic variable implies the variable could enter the basis.
If we look at change in cost, it will decrease by decrease amount multiplied by related decision variable's 
optimal value which is 1*117.

THIS IS THE LP AFTER THE CHANGES IN 2.f.:

Optimal_Logistics:
MINIMIZE
12*X_d

In [9]:
print("""Question 3. Say that Plant 0 cannot ship to distributor 0 and distributor 0 cannot ship to retailer 0.
Do you think that the total cost will increase or decrease in this case? Explain.
Model the LP and Solve regarding this. Output the basic variables and the optimal cost.""")
print("""Answer 3: We should change the related costs which are c_p_d_l[0][0] and c_d_r_l[0][0] 
by a very big number M. It can be 10000 for example. Since X_d_r_0_0 is
in the basis at optimum and by setting its cost to a very big number, it will be out of basis, 
and since X_p_d_0_0 is not in the basis and since we increase its cost, it cannot enter the basis. 
We should look at the constraint satisfaction. Our constraints are X_d_r_0_0 = 0 and X_p_d_0_0 = 0.
If the system satisfies X_p_d_0_0 = 0, but X_d_r_0_0 is 296 at optimum in the original problem.
But since there is a nonbasic variable which has a reduced cost of 0, there are a multiple optimal solutions and 
in one of them X_d_r_0_0 = 0. Thus, the optimal cost won't be affected.""")
c_p_d_l = [
[10000, 25, 14], # I set its 0th index's 0th index to a respective very big number
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[10000, 18, 17, 15], # I set its 0th index's 0th index to a respective very big number
[16, 13, 13, 10],
[15, 13, 11, 11]] 
plant_capacities = [137, 266, 104, 364]
demand_quantities = [296, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)

problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0)
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) 

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}


problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)

for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE LP AFTER THE CHANGES IN 3.:")
print("")
print(problem)
problem.solve()
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)


Question 3. Say that Plant 0 cannot ship to distributor 0 and distributor 0 cannot ship to retailer 0.
Do you think that the total cost will increase or decrease in this case? Explain.
Model the LP and Solve regarding this. Output the basic variables and the optimal cost.
Answer 3: We should change the related costs which are c_p_d_l[0][0] and c_d_r_l[0][0] 
by a very big number M. It can be 10000 for example. Since X_d_r_0_0 is
in the basis at optimum and by setting its cost to a very big number, it will be out of basis, 
and since X_p_d_0_0 is not in the basis and since we increase its cost, it cannot enter the basis. 
We should look at the constraint satisfaction. Our constraints are X_d_r_0_0 = 0 and X_p_d_0_0 = 0.
If the system satisfies X_p_d_0_0 = 0, but X_d_r_0_0 is 296 at optimum in the original problem.
But since there is a nonbasic variable which has a reduced cost of 0, there are a multiple optimal solutions and 
in one of them X_d_r_0_0 = 0. Thus, the optimal cost won't be

In [10]:
print("""Question 4. Say that the demand of retailer 0 increases.
For how much increase do you think that you will be able to satisfy the demand?
For how much increase you will lose sales?
""")
print("""Answer 4: It asks me the maximum demand of retailer 0 that i can satisfy with my current capacity constraints.
      I should get the minimum of minimum plant capacity and distributor capacities's 0th index.
      As the total demand is 728 and max supply is 871, our demand can be increased by 143 at most. That is an upper bound. 
      As the demand was 296 in the original problem, it can be at most 539.
      
      """)

c_p_d_l = [
[15, 25, 14], 
[20, 18, 11],
[14, 12, 21],
[26, 15, 28]]
c_p_r_l = [
[68, 46, 35, 58],
[57, 69, 58, 60],
[60, 69, 36, 61],
[54, 40, 49, 35]]
c_d_r_l = [
[12, 18, 17, 15], 
[16, 13, 13, 10],
[15, 13, 11, 11]] 
plant_capacities = [137, 266, 104, 364]
demand_quantities = [539, 211, 104, 117]
plants = range(4)
distributors = range(3)
retailers = range(4)

problem = LpProblem("Optimal_Logistics", LpMinimize) 
X_p_d = LpVariable.dicts("X_p_d", (plants, distributors), lowBound= 0)
X_p_r = LpVariable.dicts("X_p_r", (plants, retailers), lowBound= 0) 
X_d_r = LpVariable.dicts("X_d_r", (distributors, retailers), lowBound= 0) 

c_p_d_l_dict = makeDict((plants, distributors),c_p_d_l)
c_p_r_l_dict = makeDict((plants, retailers), c_p_r_l)
c_d_r_l_dict = makeDict((distributors, retailers), c_d_r_l)
plants_cap_dict = {plants[0]: plant_capacities[0],
                  plants[1]: plant_capacities[1],
                  plants[2]: plant_capacities[2],
                  plants[3]: plant_capacities[3]}

retailers_dem_dict = {retailers[0]: demand_quantities[0],
                      retailers[1]: demand_quantities[1],
                      retailers[2]: demand_quantities[2],
                      retailers[3]: demand_quantities[3]}


problem += lpSum(c_p_d_l[plant][distributor]*X_p_d[plant][distributor] for plant in plants for distributor in distributors)+ \
        lpSum(c_p_r_l[plant][retailer]*X_p_r[plant][retailer] for plant in plants for retailer in retailers)+ \
        lpSum(c_d_r_l[distributor][retailer]*X_d_r[distributor][retailer] for distributor in distributors for retailer in retailers)

for plant in plants:
    problem += (lpSum(X_p_r[plant][retailer] for retailer in retailers) + lpSum(X_p_r[plant][distributor] for distributor in distributors)) <= (plant_capacities[plant])
for distributor in distributors:
    problem += lpSum(X_p_d[plant][distributor] for plant in plants) == lpSum(X_d_r[distributor][retailer] for retailer in retailers)
for retailer in retailers:
    problem += lpSum(X_p_r[plant][retailer] for plant in plants) + lpSum(X_d_r[distributor][retailer] for distributor in distributors) == demand_quantities[retailer]
print("")
print("THIS IS THE LP AFTER THE CHANGES IN 4.:")
print("")
print(problem)
problem.solve()
print("Status:", LpStatus[problem.status])
print("Optimal objective value =", value(problem.objective))
print("Basic variables are:")
for var in problem.variables():
    if var.varValue >0:
        print(var.name, "=", var.varValue)
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack}
for name, c in problem.constraints.items()]
print(pandas.DataFrame(o))
for dv in problem.variables():
    print(dv.name, "'s reduced cost=", dv.dj)


Question 4. Say that the demand of retailer 0 increases.
For how much increase do you think that you will be able to satisfy the demand?
For how much increase you will lose sales?

Answer 4: It asks me the maximum demand of retailer 0 that i can satisfy with my current capacity constraints.
      I should get the minimum of minimum plant capacity and distributor capacities's 0th index.
      As the total demand is 728 and max supply is 871, our demand can be increased by 143 at most. That is an upper bound. 
      As the demand was 296 in the original problem, it can be at most 539.
      
      

THIS IS THE LP AFTER THE CHANGES IN 4.:

Optimal_Logistics:
MINIMIZE
12*X_d_r_0_0 + 18*X_d_r_0_1 + 17*X_d_r_0_2 + 15*X_d_r_0_3 + 16*X_d_r_1_0 + 13*X_d_r_1_1 + 13*X_d_r_1_2 + 10*X_d_r_1_3 + 15*X_d_r_2_0 + 13*X_d_r_2_1 + 11*X_d_r_2_2 + 11*X_d_r_2_3 + 15*X_p_d_0_0 + 25*X_p_d_0_1 + 14*X_p_d_0_2 + 20*X_p_d_1_0 + 18*X_p_d_1_1 + 11*X_p_d_1_2 + 14*X_p_d_2_0 + 12*X_p_d_2_1 + 21*X_p_d_2_2 + 26*X_p_d_3_